In [2]:
from texter.utils.config_utils import sklearn_model_config as smc
from texter.utils.config_utils import sklearn_data_config as sdc

from texter.utils.io_utils import load_config, save_config, load_model, save_model 
from texter.utils.io_utils import save_text_model, load_text_model

from texter.utils import text_utils as tu
from texter.utils.eval_utils import classifier_report

load the  hyperparams from the json file.

In [3]:
#smc_dict = dict(name="RF", config=dict(n_estimators=50, random_state=123, n_jobs=-1))
#save_config("smc_doc.json", smc_dict)
sdc_tfidf = dict(mappings_path="legal_files.csv", 
                 column="Doc_Type",  
                 root="../texter_nb/data/legal",
                 processing_type='tfidf',
                 max_features=2000,
                 limit=20000)
save_config("sdc_tfidf_doc.json", sdc_tfidf)

In [4]:
sdc_config = load_config("sdc_doc.json")
smc_config = load_config("smc_doc.json")
sdc_config_tfidf = load_config("sdc_tfidf_doc.json")

In [5]:
x_train, x_test, y_train, y_test, fitted_text_model = sdc(**sdc_config_tfidf)
model = smc(**smc_config)

classification model configured to use RandomForestClassifier algorithm.


In [23]:
save_text_model("rf_v2", fitted_text_model)

In [6]:
#sample predictions

s = """Ayahuasca emerged again in the early 1960s with the counterculture movement.\n 
Beat writers like William Burroughs, Allen Ginsberg, and Jack Kerouac all described \n
their experiences with ayahuasca, most famously in Burroughs's book The Yage Letters. """

t = "pandas is here for the day"

In [7]:
model.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 50,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': 123,
 'verbose': 0,
 'warm_start': False}

In [17]:
model.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=False, random_state=123, verbose=0, warm_start=False)

In [18]:
save_model("rf_v2", model)

In [10]:
y_pred = model.fit(x_train, y_train).predict(x_test)

In [20]:
#load saved model
rf = load_model("rf_v2")
y_pred=rf.fit(x_train, y_train).predict(x_test)

In [21]:
print(classifier_report(rf, x_test, y_test, y_pred))

1. score: 0.3333333333333333
2. classification model report:
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         0
          1       0.00      0.00      0.00         2
          2       0.50      1.00      0.67         1

avg / total       0.17      0.33      0.22         3

3. confusion matrix:
[[0 0 0]
 [1 0 1]
 [0 0 1]]


In [12]:
#vt = tu.sklearn_text_vectorizer(t, model="word2vec", 
#                                w2v_path="../../data/others/GoogleNews-vectors-negative300-SLIM.bin")

vtt = tu.sklearn_text_vectorizer(s, model="tfidf",fitted_model=fitted_text_model)
model.predict(vtt)[0]

2

In [13]:
model.predict_proba(vtt)

array([[0.26, 0.28, 0.46]])

In [14]:
#save_text_model("rf_v1_tfidf", fitted_text_model)

In [24]:
fm = tu.sklearn_text_vectorizer(t, model="tfidf", fitted_model=load_text_model("rf_v2"))

In [25]:
model.predict_proba(fm)

array([[0.3, 0.3, 0.4]])